<!--
Doc Writer email@nixdabei.de
v0.0.1, 2021-03-23
-->
[Home](../../index.ipynb) / [MicroPython auf dem Microcontroller installieren](../index.ipynb) / Croduino Esp8266 Nova: 1,3'' Display (sh1106)
***
<span style="font-size:20pt;">MicroPython auf dem Croduino Esp8266 installieren: 1,3'' Display (sh1106)</span>
***

# Flashen des Croduino Esp8266

Firmware lokal: `/030_InstallAndSetup/Esp32_NodeMCU/esp32-20220117-v1.18.bin`, oder online: [Micropython: Firmware ESP8266](https://micropython.org/download/?port=esp8266).


Vorbereitung:  
python -m pip install esptool

dann unter Windows (Port anpassen nicht vergessen):  
esptool.py --port COM4 erase_flash  
esptool.py --port COM4 --baud 460800 write_flash --flash_size=detect 0 esp8266-1m-20220117-v1.18.bin

Linux:   
esptool.py --port /dev/ttyUSB0 erase_flash  
esptool.py --port /dev/ttyUSB0 --baud 460800 write_flash --flash_size=detect 0 esp8266-1m-20220117-v1.18.bin


# Initialisierung des Croduino
## Informationen

1) Wichtige Pins:

```Python
SDA: 5
SCL: 4
On board LED (orange): 13
```
[Source: Croduino NOVA](https://e-radionica.com/en/croduino-nova.html)  
[Source: Croduino NOVA2 on GitHub](https://github.com/e-radionicacom/Croduino-NOVA2-Eagle-Files)

[Source: mjrobot.org](https://mjrobot.org/micropython-on-esp-using-jupyter-notebook)

## Schreiben von `boot.py`, `main.py` und der Module `info`.py, `sh1106`.py und `display`.py

In [1]:
# Mit dem Croduino verbinden:
#%serialconnect --port=COM3 --baud=115200
%serialconnect


#========================================================================
# Write '/info.py'
# 
# TYPE='Croduio Esp8266
#========================================================================

with open('/info.py', 'w') as f:
    f.write(r"""TYPE='Esp8266 Croduino Nova'""")

#========================================================================
# Write '/boot.py'
#========================================================================
    #This file is executed on every boot (including wake-boot from deepsleep)
    #import esp
    #esp.osdebug(None)
    #import webrepl
    #webrepl.start()
    #import uos, machine
    #uos.dupterm(None, 1) # disable REPL on UART(0)
    #import webrepl
    #webrepl.start()
    
with open('/boot.py', 'w') as f:
    f.write(r"""import gc
gc.collect()
""")

    
#========================================================================
# Write '/main.py'
#========================================================================

# On MicroPython for Esp8266  you can not give the I2C a Number
# so: i2c=I2C(1,scl=Pin(4),sda=Pin(5)) ist invalid!

with open('/main.py', 'w') as f:
    f.write(r"""from machine import Pin,I2C
import sh1106
import time
try:
 i2c=I2C(sda=Pin(5),scl=Pin(4))
 d=sh1106.SH1106_I2C(128,64,i2c)
 d.fill(0)
 d.invert(0)
 d.text("Esp8266 Croduino",1,23,1)
 d.text("OLED 128x64",20,37,1)
 d.show()
 for i in range(5):
  time.sleep_ms(200)
  d.invert(i%2)
 for i in range(45):
  d.scroll(0, -1)
  time.sleep_ms(10)
  d.show()
except:
 pass
finally:
 try: del d
 except: pass
 try: del i2c
 except: pass
 import gc
 gc.collect()""")
    


#========================================================================
# Write '/sh1106.py'
#========================================================================

with open('/sh1106.py', 'w') as f:
    f.write(r"""from micropython import const
import utime as time
import framebuf

_SET_CONTRAST        = const(0x81)
_SET_NORM_INV        = const(0xa6)
_SET_DISP            = const(0xae)
_SET_SCAN_DIR        = const(0xc0)
_SET_SEG_REMAP       = const(0xa0)
_LOW_COLUMN_ADDRESS  = const(0x00)
_HIGH_COLUMN_ADDRESS = const(0x10)
_SET_PAGE_ADDRESS    = const(0xB0)

class SH1106(framebuf.FrameBuffer):
 def __init__(self, width, height, external_vcc, rotate=0):
  self.width = width
  self.height = height
  self.external_vcc = external_vcc
  self.flip_en = rotate == 180 or rotate == 270
  self.rotate90 = rotate == 90 or rotate == 270
  self.pages = self.height // 8
  self.bufsize = self.pages * self.width
  self.renderbuf = bytearray(self.bufsize)
  if self.rotate90:
   self.displaybuf = bytearray(self.bufsize)
   super().__init__(self.renderbuf, self.height, self.width, framebuf.MONO_HMSB)
  else:
   self.displaybuf = self.renderbuf
   super().__init__(self.renderbuf, self.width, self.height, framebuf.MONO_VLSB)

  self.rotate = self.flip
  self.init_display()

 def init_display(self):
  self.reset()
  self.fill(0)
  self.poweron()
  self.flip(self.flip_en)

 def poweroff(self):
  self.write_cmd(_SET_DISP | 0x00)

 def poweron(self):
  self.write_cmd(_SET_DISP | 0x01)

 def flip(self, flag=None, update=True):
  if flag is None:
   flag = not self.flip_en
  mir_v = flag ^ self.rotate90
  mir_h = flag
  self.write_cmd(_SET_SEG_REMAP | (0x01 if mir_v else 0x00))
  self.write_cmd(_SET_SCAN_DIR | (0x08 if mir_h else 0x00))
  self.flip_en = flag
  if update:
   self.show()

 def sleep(self, value):
  self.write_cmd(_SET_DISP | (not value))

 def contrast(self, contrast):
  self.write_cmd(_SET_CONTRAST)
  self.write_cmd(contrast)

 def invert(self, invert):
  self.write_cmd(_SET_NORM_INV | (invert & 1))

 def show(self):
  (w, p, db, rb) = (self.width, self.pages, self.displaybuf, self.renderbuf)
  if self.rotate90:
   for i in range(self.bufsize):
    db[w * (i % p) + (i // p)] = rb[i]
  for page in range(self.height // 8):
   self.write_cmd(_SET_PAGE_ADDRESS | page)
   self.write_cmd(_LOW_COLUMN_ADDRESS | 2)
   self.write_cmd(_HIGH_COLUMN_ADDRESS | 0)
   self.write_data(db[(w*page):(w*page+w)])

 def reset(self, res):
  if res is not None:
   res(1)
   time.sleep_ms(1)
   res(0)
   time.sleep_ms(20)
   res(1)
   time.sleep_ms(20)

class SH1106_I2C(SH1106):
 def __init__(self, width, height, i2c, res=None, addr=0x3c, rotate=0, external_vcc=False):
  self.i2c = i2c
  self.addr = addr
  self.res = res
  self.temp = bytearray(2)
  self.temp[0] = 0x80  # Co=1, D/C#=0
  self.write_list = [b"\x40", None]  # Co=0, D/C#=1
  if res is not None:
   res.init(res.OUT, value=1)
  super().__init__(width, height, external_vcc, rotate)

 def write_cmd(self, cmd):
  self.temp[1] = cmd
  self.i2c.writeto(self.addr, self.temp)

 def write_data(self, buf):
  self.write_list[1] = buf
  self.i2c.writevto(self.addr, self.write_list)

 def reset(self):
  super().reset(self.res)""")


#========================================================================
# Write '/display.py'
#========================================================================

with open('/display.py', 'w') as f:
    f.write(r"""import sh1106
import math
import machine
class Display(sh1106.SH1106_I2C):
 def __init__(self,i2c=None):
  if i2c is None:                
   i2c=machine.I2C(sda=machine.Pin(5),scl=machine.Pin(4))
  super().__init__(128,64,i2c)
  self._x=0
  self._y=0

 def setCenter(self,x,y):
  self._x=x
  self._y=y

 def lineP(self,x,y,alpha,r0,r1,color=1):
  c=math.cos(alpha)
  s=math.sin(alpha)
  super().line(
   int(c*r0+self._x+x),
   int(s*r0+self._y+y),
   int(c*r1+self._x+x),
   int(s*r1+self._y+y),
   color
  )

 def clear(self):
  super().fill(0)

 def circle(self,x,y,r,color=1):
  pX=self._x+x
  pY=self._y+y
  x=0
  y=r
  d=3-(2*r)
  while x<=y:
   super().pixel(pX+x,pY+y,color)
   super().pixel(pX+y,pY+x,color)
   super().pixel(pX-y,pY+x,color)
   super().pixel(pX-x,pY+y,color)
   super().pixel(pX-x,pY-y,color)
   super().pixel(pX-y,pY-x,color)
   super().pixel(pX+y,pY-x,color)
   super().pixel(pX+x,pY-y,color)
   x+=1
   if d < 0:
    d=d+(4*x)+6
   else:
    d=d+4*(x-y)+10
    y-=1

 def fillCircle(self,x,y,r,color=1):
  pX=self._x+x
  pY=self._y+y
  x=0
  y=r
  d=3-(2*r)
  while x<=y:
   super().hline(pX-x,pY+y,x+x+1,color)
   super().hline(pX-y,pY+x,y+y+1,color)
   super().hline(pX-x,pY-y,x+x+1,color)
   super().hline(pX-y,pY-x,y+y+1,color)
   x+=1
   if d < 0:
    d=d+(4*x)+6
   else:
    d=d+4*(x-y)+10
    y-=1

 def pixel(self,x,y,color=None):
  if color is None:
   return super().pixel(self._x+x,self._y+y)
  else:
   super().pixel(self._x+x,self._y+y,color)

 def hline(self,x,y,width,color=1):
  super().hline(self._x+x,self._y+y,width,color)

 def vline(self,x,y,height,color=1):
  super().vline(self._x+x,self._y+y,height,color)

 def line(self,x,y,x1,y1,color=1):
  super().line(self._x+x,self._y+y,self._x+x1,self._y+y1,color)

 def rect(self,x,y,width,height,color=1):
  super().rect(self._x+x,self._y+y,width,height,color)

 def fill_rect(self,x,y,width,height,color=1):
  super().fill_rect(self._x+x,self._y+y,width,height,color)

 def text(self,str,x,y,color=1):
  super().text(str,self._x+x,self._y+y,color)

 def blit(self,fbuf,x,y,key=0):
  super().blit(fbuf,self._x+x,self._y+y,key)""")


Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


---
## [Check controller](../CheckController.ipynb)